<a href="https://colab.research.google.com/github/MarceloGarcia92/1.PersonalProject_HandModel/blob/main/Creating_Model_Crop_Images_CO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from numpy import load
from numpy import genfromtxt

In [ ]:
Dimension = '3D'
#Dimension = '2D'

Size = '160x160'
#Size = '240x240'

#Label = 'Bones'
Label = 'Joins'
#Label = 'Fingers'

Model = 'Transfer_Learning'
#Model = 'Created'

In [ ]:
Type_Model = Model + '_' + Dimension
Type_Data = Dimension + '_' + Label

path = '/content/drive/MyDrive/1. Personal Project - Marcelo Garcia/AI_Project/Hand/New_Points/' + Type_Data + '/' 
path_models = '/content/drive/MyDrive/1. Personal Project - Marcelo Garcia/AI_Project/Hand/Models/' + Type_Model + '/' 
path_x = '/content/drive/MyDrive/1. Personal Project - Marcelo Garcia/AI_Project/Hand/New_Points/3D_Joins/' 

In [ ]:
from tensorflow.keras.applications.vgg19 import VGG19

pre_trained_model = VGG19(input_shape = (160, 160, 3), 
                                include_top = False, 
                                weights = None)

for layer in pre_trained_model.layers:
    layer.trainable = True

pre_trained_model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 160, 160, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 160, 160, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 160, 160, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 80, 80, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 80, 80, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 80, 80, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 40, 40, 128)       0     

In [ ]:
import tensorflow as tf

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>=0.95):
            print("\nReached 95.0% accuracy so cancelling training!")
            self.model.stop_training = True

Expected_performance_acc = myCallback()
monitor_val_acc = tf.keras.callbacks.EarlyStopping(monitor = 'accuracy', patience = 4)

In [ ]:
last_layer = pre_trained_model.get_layer('block4_pool') 
#print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
# Flatten the output layer to 1 dimension
x = tf.keras.layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = tf.keras.layers.Dense(1052, activation='relu')(x)
# Add a dropout rate of 0.2
x = tf.keras.layers.Dropout(0.2)(x)                 
# Add a final sigmoid layer for classification
x = tf.keras.layers.Dense(2, activation='linear')(x)     

In [ ]:
model = tf.keras.Model(pre_trained_model.input, x)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 160, 160, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 160, 160, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 160, 160, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 80, 80, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 80, 80, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 80, 80, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 40, 40, 128)       0     

In [ ]:
model.compile(optimizer='adam', # change
              loss='mean_squared_logarithmic_error', 
              metrics=['accuracy'])

In [ ]:
x_train = load(path_x + 'Train/' + 'x_train_' + Dimension + '_' + Size  + '.npy')
x_validate = load(path_x + 'Validate/' + 'x_validate_' + Dimension + '_' + Size + '.npy')
x_test = load(path_x + 'Test/' +  'x_test_' + Dimension + '_' + Size + '.npy')

In [ ]:
def Create_model(points, epochs):
  saved_list = []
  for point in range(0, len(points)):
    
    train = genfromtxt(path + 'Train/' + points[point] + '_train' + '_' + Dimension + '_' + Size + '.csv', delimiter=",")
    validate = genfromtxt(path + 'Validate/' + points[point] +'_Validate' + '_' + Dimension + '_' + Size + '.csv', delimiter=",")
    test = genfromtxt(path + 'Test/' + points[point] + '_Test' + '_' + Dimension + '_' + Size + '.csv', delimiter=",")

    model_bone = tf.keras.Model(pre_trained_model.input, x)
    model_bone.summary()

    model_bone.compile(optimizer='adam', # change
              loss='mean_squared_logarithmic_error', 
              metrics=['accuracy'])

    bone_history = model_bone.fit(x_train, train, epochs=epochs, batch_size= 128, verbose= 1, validation_data=(x_validate, validate), callbacks = [monitor_val_acc]) #, Expected_performance_acc

    [z, y] = model_bone.evaluate(x_test, test,verbose=1)

    model_bone.save(path_models  + points[point] + '_model_' + Type_Model + '_' + Size + '2.h5')
    print(points[point] + '_model save')
    saved_list.append(points[point])

  print(saved_list)

In [ ]:
list_joins = ['cero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven', 'twelve', 'thirteen', 'fourteen', 'fifteen', 'sixteen', 'seventeen', 'eighteen', 'nineteen', 'twenty']
list_bones = ['Carpals_1', 'Metacarpals_1', 'Proxima_phalanges_1', 'Distal_phalanges_1', 'Metacarpals_2', 'Proxima_phalanges_2', 'Intermediate_phalanges_2', 'Distal_phalanges_2', 'Metacarpals_3', 'Proxima_phalanges_3', 'Intermediate_phalanges_3', 'Distal_phalanges_3', 'Metacarpals_4', 'Proxima_phalanges_4', 'Intermediate_phalanges_4', 'Distal_phalanges_4', 'Metacarpals_5', 'Proxima_phalanges_5', 'Intermediate_phalanges_5', 'Distal_phalanges_5']
list_fingers = ['Thumb', 'Index_finger', 'Middle_finger', 'Ring_finger', 'Pinky']

In [ ]:
Create_model(list_joins, 50)